# 5．ノンパラメトリック検定 ｜Pythonプログラミングを使った統計分析の基礎

### [Logics of Blue](https://logics-of-blue.com/) 馬場真哉：2019年3月1日

<p>
    このファイルの内容は、スライドでは説明しません。<br>
    時間に余裕がある方が補足的に参照することを想定しています。
</p>

## 目的
- Mann-WhitenyのU検定を通して、ノンパラメトリック検定の基本を学ぶ
    - ノンパラメトリック検定の基本的な考え方
    - Mann-WhitenyのU検定の考え方
    - Pythonによる検定の実行
 
## 目次
1. [パラメトリックとノンパラメトリック](#1．パラメトリックとノンパラメトリック)
2. [Mann-WhitenyのU検定の基本](#2．Mann-WhitenyのU検定の基本)
3. [Mann-WhitenyのU検定の実行](#3．Mann-WhitenyのU検定の実行)

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 表示桁数の指定
%precision 3
# グラフをjupyter Notebook内に表示させるための指定
%matplotlib inline

# 不要なワーニングを出力させない
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

***
## 1．パラメトリックとノンパラメトリック

<p>
    ここでは、二項検定やt検定といったパラメトリックな検定方法と対比して<br>
    ノンパラメトリックな検定の方法を紹介します。
</p>

<p>
    まずは、パラメトリックであるとかノンパラメトリックという言葉の意味を学びます
</p>


#### パラメトリック
<p>
    パラメトリックな方法の場合は、母集団に対して、二項分布や正規分布と言った、何らかの確率分布を想定します。<br>
    二項分布の確率質量関数を使うと、試行回数Nと成功確率pという2つのパラメタが与えられれば、成功回数mの確率分布が即座に計算できますね。<br>
    成功確率といった「少数のパラメタ」が与えられれば、即座に確率分布が得られるのが、確率質量関数を使う大きなメリットです。
</p>

<p>
    しかし、母集団の確率分布に対して、例えば二項分布であるとか正規分布と言った仮定を置くことができないこともあります。<br>
    その時はパラメトリックな方法が使えないので、ノンパラメトリックな方法を使います。
</p>

#### ノンパラメトリック
<p>
    ノンパラメトリックは「少数のパラメタだけわかればいい」という考え方をとらない方法論です。<br>
    二項分布や正規分布という便利な数式は使わず、順列組合せの公式だけを使って、p値等を計算し、統計的仮説検定を行います。
</p>


***
## 2．Mann-WhitenyのU検定の基本

<p>
    ここでは、2標本のt検定のノンパラメトリック検定バージョンともいえる、Mann-WhitenyのU検定を学びます。
</p>

### 順列組合せの復習

<p>
    母集団に、二項分布や正規分布を想定しない代わりに、順列組合せの計算を頻繁に活用します。<br>
    ここで、順列組合せの公式を復習しておきます。<br>
    ただし、Mann-WhitenyのU検定を理解するのに必要な公式しか紹介しません。
</p>


#### 並び替え問題
<p>
    A,B,C,Dの4つの品物があります。これを並び替えた時、何通りの場合の数があるでしょうか。<br>
    1つ目を選ぶ方法は4通り。例えばBを選んだとします。残りはA,C,Dの3つだけです。<br>
    2つ目を選ぶ方法は3通りしかありません。すでに1つ選ばれた後だからですね。<br>
    3つ目を選ぶ方法は2通り。<br>
    4つ目を選ぶ方法は1通りしかありません。
</p>

<p>
    というわけで、A,B,C,Dの4つの品物の並び替えは、<br>
    4×3×2×1<br>
    で、その場合の数が計算されます。<br>
    1つずつ値を減らして掛け合わせていく計算を、階乗と呼び、例えば「4！」のように書きます。<br>
    Pythonにおいては、mathライブラリをインポートしてから、math.factorial関数を実行します。
</p>

In [2]:
# 階乗の計算
import math
math.factorial(4)

24

#### 2種類の品物の並び替え問題
<p>
    AとBの2つの品物があります。商品Aは3つ、Bは2つあります。これを並び替えた時、何通りの場合の数があるでしょうか。<br>
    5個あるから5の階乗としたいところです。しかし商品Aは同じ物が3つあるので、見分けがつきません。<br>
    見分けがつかない分だけ、場合の数を少なくします。<br>
    具体的には、商品Aだけの並び順（3！）と商品Bだけの並び順（2！）で割ってやります。
</p>

In [3]:
math.factorial(5) / (math.factorial(3) * math.factorial(2))

10.000

### 順列組み合わせ問題の、検定への応用
<p>
    魚Aと魚Bで大きさが有意に異なるか否かを判別することを考えます。
</p>

#### 2種類の魚の並び替え問題
<p>
    魚Aが2尾、魚Bが2尾いたとします。<br>
    この時、魚の並び順は、6通りあるはずです。
</p>

In [4]:
math.factorial(4)/(math.factorial(2) * math.factorial(2))

6.000

#### 魚Bの方が大きいように見えるのは偶然か？：パターン1
<p>
    魚Aの体長組成が[1cm、2cm]だったとします。<br>
    魚Bの体長組成が[3cm、4cm]だったとします。<br>
</p>

<p>
    魚Aは、すべて魚Bよりも小さくなっていますね。<br>
    このようなパターンは1パターンしかないので、<br>
    『魚Aが魚Bよりも常に小さな魚に、偶然なってしまう確率は、1÷6＝1/6である』と計算されます。<br>
    この1/6がp値です。<br>
    両側検定をする場合は、2で掛けて、2/6がp値となります。
</p>


#### 魚Bの方が大きいように見えるのは偶然か？：パターン2
<p>
    魚Aの体長組成が[1cm、3cm]だったとします。<br>
    魚Bの体長組成が[2cm、4cm]だったとします。<br>
</p>

<p>
    魚Aの1cmの魚は、魚Bのどの魚よりも小さいです。<br>
    一方、3cmの魚は、魚B「2cmの魚」よりは大きく、「4cmの魚」よりは小さいです。<br>
    このため、パターン1よりも『あまり、魚Bの方が大きいように見えない』といえます。
</p>

<p>
   上記の体長組成となる確率はやはり1/6です。<br>
   すると『今回のデータよりも、より極端に、魚Bの方が大きく見える確率』は<br>
    パターン1の1/6と、パターン2の1/6を足して、2/6となります。この2/6がp値です。<br>
    両側検定をする場合は、p値を2倍して、4/6がp値となります。
</p>


### Mann-WhitenyのU検定の基本

<p>
    Mann-WhitenyのU検定の手順を以下に記します。
</p>

<p>
    Step1. ありうる体長組成の場合の数を計算します。<br>
    Step2. 今回のデータと同じか、それ以上に『極端にどちらかが大きく見える場合の数』を計算します。<br>
    Step3. 『極端にどちらかが大きく見える場合の数』 ÷ 『ありうる体長組成の場合の数』を計算します。これがp値です。<br>
    Step4. 両側検定をする場合は、p値を2倍します。
</p>


<p>
    scipy.statsのmannwhitneyu関数を適用することで実行できます。<br>
    しかし、この関数はp値の計算を簡略化しています。場合の数をいちいち計算するのは大変なので、近似計算をしているのです。<br>
    そのため、理論上の値と少しずれます。<br>
    サンプルサイズが20以上の時に使うことが奨励されています。
</p>

***
## 3．Mann-WhitenyのU検定の実行

<p>
    Mann-WhitenyのU検定を実際に行ってみます。<br>
    t検定との比較も行います。
</p>

In [5]:
# 検定対象となるデータの作成
data_x = np.arange(1,21,1)
data_y = np.arange(4,24,1)

print("data_x:",data_x)
print("data_y:",data_y)

data_x: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
data_y: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [6]:
# Mann-WhitenyのU検定
stats.mannwhitneyu(data_x, data_y, alternative='two-sided')

MannwhitneyuResult(statistic=144.5, pvalue=0.13650248074290666)

In [7]:
# t検定
stats.ttest_ind(data_x, data_y, equal_var = False)

Ttest_indResult(statistic=-1.6035674514745464, pvalue=0.11709007587900368)

In [8]:
# データを1つだけ値を変えてみる
data_y[19] = 35

print("data_x:",data_x)
print("data_y:",data_y)

data_x: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
data_y: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 35]


In [9]:
# Mann-WhitenyのU検定は、p値は変わらない
stats.mannwhitneyu(data_x, data_y, alternative='two-sided')

MannwhitneyuResult(statistic=144.5, pvalue=0.13650248074290666)

In [10]:
# t検定はp値が微妙に変わる
stats.ttest_ind(data_x, data_y, equal_var = False)

Ttest_indResult(statistic=-1.7046494057173622, pvalue=0.09680187034992213)

#### まとめ
<p>
    ノンパラメトリック検定は、外れ値に対して頑健だと言われます。これは、データの大小の順番だけにこだわったおかげでしょう。<br>
    ただし、ノンパラメトリック検定といえども万能ではありません。<br>
    例えば不等分散のデータに対しては、t検定の方が適しているという報告もあります。<br>
    データに合わせて、検定の方法を適宜変えるようにしてください。
</p>